In [1]:
from LandBase import *

# 법정동 아파트 목록을 DB에 저장 (출처: 공공데이터포탈)
def aptlist_into_db(p_con, p_lawd_cd, p_ignore_flag=True) : # p_ignore_flag = 기존에 데이터가 존재할 때 무시할지 여부
    table_name = 'kapt_list'
    exist_flag = False
    
    # 기존 데이터를 무시하지 말라(p_ignore_flag==False)는 옵션이 주어지면
    # 같은 데이터가 이미 있는지 검사해서 있으면 함수를 종료함
    if (p_ignore_flag==False) :
        exist_flag = check_data_exists(p_con, 'select 1 as cnt from '+table_name+' where lawd_cd = '+p_lawd_cd+' limit 1')
        if (exist_flag) :
            return 
    
    # Open API 호출
    url = 'http://apis.data.go.kr/1611000/AptListService/getLegaldongAptList' \
    + '?loadCode={lawd_cd}&pageNo=1&numOfRows=300&ServiceKey={key}'
    url = url.format(lawd_cd=p_lawd_cd, key=data_go_kr_apikey)
    response = requests.get(url)
    # xml 형식으로 반환된 Open API 결과를 DataFrame으로 변환 + 법정동 정보 추가
    df_kapt = xml2df(response.content, p_element_name='items', p_level=2) 
    df_kapt.insert(0, 'lawd_cd', p_lawd_cd)
    df_kapt.rename(columns={'kaptCode':'kapt_cd', 'kaptName':'kapt_nm'}, inplace=True)

    # DB 처리
    con.execute('delete from ' + table_name + ' where lawd_cd = ' + p_lawd_cd)
    df_kapt.to_sql(name=table_name, con=p_con,  if_exists='append', index=False, chunksize=1000)

# 법정동 아파트 기본 정보를 DB에 저장 (출처: 공공데이터포탈)
def aptinfo_into_db(p_con, p_kapt_cd, p_ignore_flag=True) : # p_ignore_flag = 기존에 데이터가 존재할 때 무시할지 여부
    table_name = 'kapt_info'
    exist_flag = False
    
    # 기존 데이터를 무시하지 말라(p_ignore_flag==False)는 옵션이 주어지면
    # 같은 데이터가 이미 있는지 검사해서 있으면 함수를 종료함
    if (p_ignore_flag==False) :
        exist_flag = check_data_exists(p_con, 'select 1 as cnt from '+table_name+' where kapt_cd = \''+p_kapt_cd+'\' limit 1')
        if (exist_flag) :
            return 
    
    # Open API 호출
    url = 'http://apis.data.go.kr/1611000/AptBasisInfoService/getAphusBassInfo?kaptCode={kapt_cd}&ServiceKey={key}'
    url = url.format(kapt_cd=p_kapt_cd, key=data_go_kr_apikey)
    response = requests.get(url)
    # xml 형식으로 반환된 Open API 결과를 DataFrame으로 변환
    df_kapt = xml2df(response.content, p_element_name='item', p_level=1) 
    rm_list = ['hoCnt','kaptFax', 'kaptTel','kaptUrl']
    rm_columns = [c for c in rm_list if c in df_kapt.columns]
    df_kapt.drop(rm_columns, axis=1, inplace=True)
    df_kapt.rename(columns={'bjdCode':'lawd_cd', \
                            'kaptCode':'kapt_cd', \
                            'kaptName':'kapt_nm', \
                            'kaptdaCnt':'kapt_da_cnt', \
                            'kaptDongCnt':'kapt_dong_cnt', \
                            'codeAptNm':'code_apt_nm', \
                            'codeHallNm':'code_hall_nm', \
                            'codeHeatNm':'code_heat_nm', \
                            'codeSaleNm':'code_sale_nm', \
                            'kaptUsedate':'kapt_use_date', \
                            'kaptMparea_60':'kapt_marea_60', \
                            'kaptMparea_85':'kapt_marea_85', \
                            'kaptMparea_135':'kapt_marea_135', \
                            'kaptdaSize_136':'kapt_marea_136', \
                            'kaptMarea':'kapt_marea', \
                            'kaptTarea':'kapt_tarea', \
                            'privArea':'priv_area', \
                            'kaptAddr':'kapt_addr', \
                            'doroJuso':'doro_juso', \
                            'kaptAcompany':'kapt_acompany', \
                            'kaptBcompany':'kapt_bcompany', \
                            'codeMgrNm':'code_mgr_nm'}, \
                inplace=True)

    # DB 처리
    con.execute('delete from ' + table_name + ' where kapt_cd = \'' + p_kapt_cd + '\'')
    df_kapt.to_sql(name=table_name, con=p_con,  if_exists='append', index=False, chunksize=1000)


In [ ]:
# DB Connection Open
con = get_db_connection()

# 공동주택목록을 DB에 입력
# 법정동 정보를 DB에서 하나씩 읽어와서 처리
# 입력대상테이블: kapt_list 
sql = 'select lawd_cd from lawd_mstr'
lawd_list = con.execute(sql)
for lawd_cd in lawd_list :
    aptlist_into_db (p_con=con, p_lawd_cd=lawd_cd[0], p_ignore_flag=False)

# 공동주택 기본 정보를 DB에 입력
# 아파트코드 정보를 DB에서 하나씩 읽어와서 처리
# 입력대상테이블: kapt_info
sql = 'select kapt_cd from kapt_list order by lawd_cd, kapt_cd'
kapt_list = con.execute(sql)
for kapt_cd in kapt_list :
    aptinfo_into_db (p_con=con, p_kapt_cd=kapt_cd[0], p_ignore_flag=False)

# DB Connection Close
disconnect_db(con)

print('Program Finised')